In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

In [2]:
# Import libraries
import pandas as pd
import os

from src.data_wrangling import *

In [3]:
# Set working directory
os.chdir('../')

In [4]:
# Load the data
aup = pd.read_csv('resources/raw_data/data.tsv', sep='\t', index_col=0)
mdata = pd.read_csv('resources/raw_data/metadata.tsv', sep='\t')

In [5]:
aup

,b1790p079_PP1_MCF7_AZD1480_r1,b1790p079_PP1_MCF7_AZD1480_r2,b1790p079_PP1_MCF7_AZD5363_r1,b1790p079_PP1_MCF7_AZD5363_r2,b1790p079_PP1_MCF7_BX_r1,b1790p079_PP1_MCF7_BX_r2,b1790p079_PP1_MCF7_CHIR_r1,b1790p079_PP1_MCF7_CHIR_r2,b1790p079_PP1_MCF7_DMSO_r1,b1790p079_PP1_MCF7_DMSO_r2,...,b1790p077_PP2_MCF7_KU_r1,b1790p077_PP2_MCF7_KU_r2,b1790p077_PP2_MCF7_PD_r1,b1790p077_PP2_MCF7_PD_r2,b1790p077_PP2_MCF7_PF_r1,b1790p077_PP2_MCF7_PF_r2,b1790p077_PP2_MCF7_TORIN_r1,b1790p077_PP2_MCF7_TORIN_r2,b1790p077_PP2_MCF7_TRAMETINIB_r1,b1790p077_PP2_MCF7_TRAMETINIB_r2
Substrate,,,,,,,,,,,,,,,,,,,,,
1A24_HUMAN(S356),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1A24_HUMAN(S359),2.692968e+08,3.502752e+08,2.339065e+08,1.473238e+08,2.292088e+08,1.992612e+08,2.226755e+08,2.031111e+08,1.724312e+08,1.952799e+08,...,1.942673e+08,2.446224e+08,1.228907e+08,2.781818e+08,2.341428e+08,1.885402e+08,1.601014e+08,2.877380e+08,4.610970e+08,4.607186e+08
1B39_HUMAN(M1),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
1B39_HUMAN(M4),7.695947e+08,5.231202e+08,1.278819e+09,7.016728e+08,6.317868e+07,3.288562e+08,7.672546e+08,8.864559e+08,3.021734e+09,1.579953e+09,...,0.000000e+00,0.000000e+00,5.279555e+08,1.183298e+09,1.879283e+09,0.000000e+00,1.289702e+09,1.528154e+09,9.917060e+08,1.658262e+08
AAAS(S495),2.664712e+09,3.635279e+09,3.285959e+09,2.743536e+09,3.822861e+09,4.393195e+09,3.287022e+09,3.540691e+09,2.247902e+09,6.012226e+09,...,8.963979e+09,8.351130e+09,9.330759e+09,7.767012e+09,7.900387e+09,7.211204e+09,8.041161e+09,8.921806e+09,8.499730e+09,7.352127e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX(T274),8.974883e+07,1.260209e+08,9.165969e+07,1.151805e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.608385e+07,4.336183e+07,...,0.000000e+00,5.477656e+07,1.636384e+07,4.403086e+07,3.838972e+07,5.303532e+07,8.103282e+07,9.077570e+07,0.000000e+00,1.604186e+07
ZZEF1(S1464),2.893030e+08,2.244606e+08,0.000000e+00,1.258947e+08,4.972942e+08,7.212378e+08,3.284692e+08,2.707639e+08,7.309032e+08,3.075764e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.685274e+08,4.616440e+08,6.383198e+07,0.000000e+00,1.184771e+08,1.970004e+08
ZZEF1(S1518),4.819850e+08,4.258855e+08,4.693328e+08,5.905167e+08,1.361086e+09,1.475927e+09,1.257499e+09,1.364008e+09,2.441649e+09,1.273486e+09,...,8.518449e+08,1.137797e+09,7.286048e+08,8.506918e+08,1.302619e+09,1.115332e+09,8.928796e+08,1.064749e+09,6.994471e+08,8.058096e+08


In [6]:
mdata

,name,perturbagen,cell_line,batch,rep,sampleID,status
0,b1790p087_MCF7_AC220_PP1_r1,AC220,MCF7,b1790p087,1,b1790p087_MCF7_AC220_1,perturbed
1,b1790p087_MCF7_AC220_PP1_r2,AC220,MCF7,b1790p087,2,b1790p087_MCF7_AC220_2,perturbed
2,b1790p087_MCF7_AC220_PP2_r1,AC220,MCF7,b1790p087,3,b1790p087_MCF7_AC220_3,perturbed
3,b1790p087_MCF7_AC220_PP2_r2,AC220,MCF7,b1790p087,4,b1790p087_MCF7_AC220_4,perturbed
4,b1790p086_MCF7_MP-470_PP1_r1,Amuvatinib,MCF7,b1790p086,1,b1790p086_MCF7_Amuvatinib_1,perturbed
...,...,...,...,...,...,...,...
271,b1790p085_MCF7_ULIXERTINIB_PP2_r2,Ulixertinib,MCF7,b1790p085,4,b1790p085_MCF7_Ulixertinib_4,perturbed
272,b1790p091_MCF7_Vemurafenib_PP1_r1,Vemurafenib,MCF7,b1790p091,1,b1790p091_MCF7_Vemurafenib_1,perturbed
273,b1790p091_MCF7_Vemurafenib_PP1_r2,Vemurafenib,MCF7,b1790p091,2,b1790p091_MCF7_Vemurafenib_2,perturbed
274,b1790p091_MCF7_Vemurafenib_PP2_r1,Vemurafenib,MCF7,b1790p091,3,b1790p091_MCF7_Vemurafenib_3,perturbed


In [7]:
# Rename samples (optional)
aup = rename_samples_based_on_metadata(aup, mdata)

In [8]:
# Remove non S/T/Y residues
aup_f = filter_canonical_psts(aup)

In [9]:
# Merge duplicated features
aup_fd = merge_duplicates(aup_f)

In [10]:
# Log2-transform data
aup_fdt = log2_transform_and_clean(aup_fd)

/Users/magdalena/miniforge3/envs/magpipe/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log2
  result = func(self.values, **kwargs)


In [11]:
aup_fdt

name,b1790p087_MCF7_AC220_1,b1790p087_MCF7_AC220_2,b1790p087_MCF7_AC220_3,b1790p087_MCF7_AC220_4,b1790p086_MCF7_Amuvatinib_1,b1790p086_MCF7_Amuvatinib_2,b1790p086_MCF7_Amuvatinib_3,b1790p086_MCF7_Amuvatinib_4,b1790p091_MCF7_AT13148_1,b1790p091_MCF7_AT13148_2,...,b1790p085_MCF7_U73122_3,b1790p085_MCF7_U73122_4,b1790p085_MCF7_Ulixertinib_1,b1790p085_MCF7_Ulixertinib_2,b1790p085_MCF7_Ulixertinib_3,b1790p085_MCF7_Ulixertinib_4,b1790p091_MCF7_Vemurafenib_1,b1790p091_MCF7_Vemurafenib_2,b1790p091_MCF7_Vemurafenib_3,b1790p091_MCF7_Vemurafenib_4
Substrate,,,,,,,,,,,,,,,,,,,,,
1A24_HUMAN(S356),25.870812,NaN,26.254517,26.223129,26.351009,NaN,NaN,26.245589,26.428473,26.114802,...,27.382411,27.373923,27.326930,27.453267,NaN,27.688209,26.306061,25.682014,25.246768,NaN
1A24_HUMAN(S359),25.870812,NaN,26.254517,26.223129,26.351009,NaN,NaN,26.245589,26.428473,26.114802,...,27.382411,27.373923,27.326930,27.453267,NaN,27.688209,26.306061,25.682014,25.246768,NaN
AAAS(S495),32.896346,32.739687,32.303542,32.262632,31.360080,31.353708,31.260036,31.327017,32.427601,32.122839,...,31.892032,31.723375,31.421182,31.767499,31.576630,32.058853,32.304805,32.031872,32.068424,32.020643
AAGAB(S310),28.801965,28.723032,29.125469,29.284707,27.913905,27.922498,26.890910,27.763465,25.475470,25.570484,...,NaN,NaN,28.943441,29.007726,25.700312,NaN,NaN,24.831618,NaN,25.668625
AAGAB(S311),29.603953,29.506551,29.684819,30.093038,27.913905,27.922498,26.890910,27.763465,26.120034,26.071147,...,NaN,NaN,28.943441,29.007726,25.700312,NaN,25.403631,26.068589,NaN,26.269990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX(T274),25.303563,26.035575,25.840835,25.990198,NaN,26.362208,25.718646,NaN,25.084586,24.126621,...,24.570957,25.820922,NaN,NaN,NaN,24.646844,24.530566,NaN,25.149575,25.436921
ZZEF1(S1464),28.789735,29.058235,27.409050,28.412225,27.278065,27.884694,NaN,27.548852,NaN,NaN,...,29.101896,29.350810,29.458305,29.479319,29.442303,29.110623,NaN,27.092905,NaN,NaN
ZZEF1(S1518),29.155195,29.098349,29.584911,29.838893,29.554872,29.248563,29.321634,29.365608,30.331247,30.521423,...,30.445500,30.386711,29.561183,29.679949,29.964833,29.988700,29.516100,29.345292,29.252827,29.122827
